In [223]:
import pandas as pd

activitats = pd.read_csv('activitats.csv', encoding = 'latin-1')
notes = pd.read_csv('notes.csv', sep=';', encoding = 'latin-1')
notes.rename(columns = {notes.columns[0]: 'userid'}, inplace = True)
trameses = pd.read_csv('trameses.csv', sep=',', encoding = 'latin-1')
trameses.rename(columns = {trameses.columns[0]: 'tramesa_id'}, inplace = True)

In [224]:
notes_aules = notes['aula_id'].unique()
activitats_aules = activitats['aula_id'].unique()
print(notes_aules,activitats_aules)

[ 92 143 176  87 141 184] [ 87  92 114 125 126 141 143 164 165 170 176 184]


Eliminamos los registros de las actividades de las asignatura de las cuales no tenemos notas y aquellos que contengan "PRU", "PROVA" o "OCULT" ya que són pruebas de los docentes.

In [225]:
activitats = activitats[~activitats["aula_id"].isin([114, 125, 126, 164, 165, 170])]
activitats = activitats[~activitats["activitat"].str.contains('PROVA|PRU|OCULT', case=False)]
activitats

,activitat_id,activitat,aula_id,startdate,duedate,grade
0,3,Problema 1.1 - Hello world!!!,87,0,0,100
1,4,Problema 1.2 - Hello world!!! ++,87,0,0,100
2,5,Problema 3a.1: Càlcul edat,87,0,0,100
3,6,Problema 3a.8: Conversió dòlars a euros,87,0,0,100
4,7,Problema 3a.11: Mitjana de notes,87,0,0,100
...,...,...,...,...,...,...
1793,4058,Problema 8.1: Classe Point,184,0,0,100
1794,4059,Problema 8.2: Classe Rectangle,184,0,0,100
1796,4194,(OPCIONAL) Problema 9.24 Triple pitagòric,184,0,0,100
1799,4386,Lliurament Pràctica 5: NEW La Major,176,1571985000,1576745700,100


In [226]:
finals_nan_count = notes.groupby('aula_id').agg(
    total_rows=('F_Grade', 'size'),  # Cuenta el total de filas en cada grupo,
    count_F_NaNs=('F_Grade', lambda x: x.isna().sum())  # Cuenta valores NaN en 'F_Grade'
)

print(finals_nan_count)

         total_rows  count_F_NaNs
aula_id                          
87              112             0
92              252             1
141             113             3
143             238             4
176             248           248
184             125           125


Tenemos 6 asignaturas y podemos eliminar  2 de ellas, ya que aun no se han realizado los exámenes.

In [227]:
notes = notes[~notes["aula_id"].isin([176, 184])]
activitats = activitats[~activitats["aula_id"].isin([176, 184])]

In [228]:
finals_nan_count = notes.groupby('aula_id').agg(
    total_rows=('F_Grade', 'size'),  
    count_F_NaNs=('F_Grade', lambda x: x.isna().sum())  
)

print(finals_nan_count)

         total_rows  count_F_NaNs
aula_id                          
87              112             0
92              252             1
141             113             3
143             238             4


Eliminamos los 8 registros que no tienen nota final

In [229]:
notes = notes[~notes["F_Grade"].isna()]

Contamos el número de actividades por cada asignatura

In [230]:
actxasig = activitats.groupby('aula_id').agg(
    total_rows=('aula_id', 'size')
)

print(actxasig)

         total_rows
aula_id            
87              194
92              141
141             243
143             171


Comprobamos si quedan nans por tratar

In [231]:
nan_counts = activitats.isnull().sum()
nan_counts

activitat_id    0
activitat       0
aula_id         0
startdate       0
duedate         0
grade           0
dtype: int64

In [232]:
nan_counts = notes.isnull().sum()
nan_counts

userid            0
aula_id           0
P_Grade         486
P_Grade_Date    485
F_Grade           0
F_Grade_Date      0
R_Grade         462
R_Grade_Date      0
dtype: int64

In [233]:
finals_nan_count = notes.groupby('aula_id').agg(
    total_rows=('aula_id', 'size'),  
    P_NaNs=('P_Grade', lambda x: x.isna().sum()),
    R_NaNs=('R_Grade', lambda x: x.isna().sum())
)

print(finals_nan_count)

         total_rows  P_NaNs  R_NaNs
aula_id                            
87              112       0      76
92              251     251     170
141             110       1      62
143             234     234     154


Observamos que aula_id 92 y 143 no hacen parciales.

In [234]:
notes = notes[~((notes["aula_id"] == 141) & (notes["P_Grade"].isna()))]

In [235]:
timestamp_min_87 = notes[notes["aula_id"]==87]['F_Grade_Date'].min()
timestamp_max_87 = notes[notes["aula_id"]==87]['F_Grade_Date'].max()
timestamp_min_92 = notes[notes["aula_id"]==92]['F_Grade_Date'].min()
timestamp_max_92 = notes[notes["aula_id"]==92]['F_Grade_Date'].max()
timestamp_min_141 = notes[notes["aula_id"]==141]['F_Grade_Date'].min()
timestamp_max_141 = notes[notes["aula_id"]==141]['F_Grade_Date'].max()
timestamp_min_143 = notes[notes["aula_id"]==143]['F_Grade_Date'].min()
timestamp_max_143 = notes[notes["aula_id"]==143]['F_Grade_Date'].max()

In [236]:
fecha_min_1 = pd.to_datetime(timestamp_min_87, unit='s')
fecha_max_1 = pd.to_datetime(timestamp_max_87, unit='s')
fecha_min_2 = pd.to_datetime(timestamp_min_92, unit='s')
fecha_max_2 = pd.to_datetime(timestamp_max_92, unit='s')
fecha_min_3 = pd.to_datetime(timestamp_min_141, unit='s')
fecha_max_3 = pd.to_datetime(timestamp_max_141, unit='s')
fecha_min_4 = pd.to_datetime(timestamp_min_143, unit='s')
fecha_max_4 = pd.to_datetime(timestamp_max_143, unit='s')


print("Fecha 87:", fecha_min_1, fecha_max_1)
print("Fecha 92:", fecha_min_2, fecha_max_2)
print("Fecha 141:", fecha_min_3, fecha_max_3)
print("Fecha 143:", fecha_min_4, fecha_max_4)

Fecha 87: 2023-01-17 14:00:00 2023-01-19 14:00:00
Fecha 92: 2022-11-07 14:00:00 2022-11-07 14:00:00
Fecha 141: 2024-01-07 14:00:00 2024-01-16 14:00:00
Fecha 143: 2023-11-13 14:00:00 2023-11-13 14:00:00


Se deduce que 87 y 141 son la misma asignatura, así como 92 y 143.


Por lo tanto separamos el dataset en 2 con el objetivo de hacer un modelo para cada asignatura.

In [237]:
notes = notes.drop(columns=["R_Grade", "R_Grade_Date", "P_Grade_Date"], axis=1)
notes

,userid,aula_id,P_Grade,F_Grade,F_Grade_Date
0,176,92,NaN,"9,25",1667829600
1,179,92,NaN,"7,75",1667829600
2,168,92,NaN,"7,75",1667829600
3,401,92,NaN,"3,50",1667829600
4,482,92,NaN,"7,00",1667829600
...,...,...,...,...,...
958,1642,141,"4,75","2,9",1705413600
959,1633,141,"7,6","5,1",1705413600
960,1656,141,"5,88","8,04",1705413600
961,2043,141,"7,8",2,1705413600


In [238]:
notes_A = notes[notes["aula_id"].isin([87, 141])]
notes_B = notes[~notes["aula_id"].isin([87, 141])]
notes_B = notes_B.drop(columns=["P_Grade"], axis=1)

In [239]:
activitats_A = activitats[activitats["aula_id"].isin([87, 141])]
activitats_B = activitats[~activitats["aula_id"].isin([87, 141])]

In [240]:
coinc_A = activitats_A.groupby('activitat').agg(
    total_rows=('activitat', 'size')
)
coinc_A

,total_rows
activitat,
Problema 8.11: Point2D i Point3D,2
Problema 8.14: Baralla de cartes,1
(OPCIONAL) Problema 3b.1: És de l'interval?,2
(OPCIONAL) Problema 3b.20: Iguals o diferents?,2
(OPCIONAL) Problema 3b.33: Zodíac xinès,2
...,...
Recuperació IP - Exercici 3 (1 punt),1
Recuperació IP - Exercici 4 (1 punt),1
Recuperació IP - Exercici 5 (1 punt),1


In [241]:
coinc_B = activitats_B.groupby('activitat').agg(
    total_rows=('activitat', 'size')
)
coinc_B

,total_rows
activitat,
(OPCIONAL) Problema 3b.1: És de l'interval?,2
(OPCIONAL) Problema 3b.20: Iguals o diferents?,2
(OPCIONAL) Problema 3b.33: Zodíac xinès,2
(OPCIONAL) Problema 3b.37: Calculadora,2
(OPCIONAL) Problema 3b.3: Positiu o negatiu?,2
...,...
RECUPERACIÓ Entrega Pràctica 3 - 4: MasterMind,1
RECUPERACIÓ Entrega Pràctica 5: BlackJack,2
RECUPERACIÓ Entrega Pràctica 5: Encriptació,2


Finalmente nos decantamos por hacer un modelo para cada asignatura para simplificar el proceso, ya que aunque sean la misma asignatura, cada año se modifican el numero de entregas.

In [242]:
activitats_x_aula = {}
notes_x_aula = {}

for aula_id in activitats['aula_id'].unique():
    df_act = activitats[activitats['aula_id'] == aula_id]
    df_notes = notes[notes['aula_id'] == aula_id]
    
    activitats_x_aula[aula_id] = df_act
    notes_x_aula[aula_id] = df_notes


In [244]:
print(activitats_x_aula[87])

     activitat_id                                          activitat  aula_id  \
0               3                      Problema 1.1 - Hello world!!!       87   
1               4                   Problema 1.2 - Hello world!!! ++       87   
2               5                         Problema 3a.1: Càlcul edat       87   
3               6            Problema 3a.8: Conversió dòlars a euros       87   
4               7                   Problema 3a.11: Mitjana de notes       87   
..            ...                                                ...      ...   
207           214                  Problema x.32: Mínim comú divisor       87   
208           215                         Problema 8.1: Classe Point       87   
209           216                     Problema 8.2: Classe Rectangle       87   
499          1652        RECUPERACIO - Lliurament Pràctica 2: Hotels       87   
859          2221  RECUPERACIÓ:  Practica 1: Rock, Paper, Scissor...       87   

      startdate     duedate